In [1]:
from prisoner_guard_prompter import env

parallel_env = env(render_mode="human", max_cycles=1000)
observations, infos = parallel_env.reset()

pygame 2.3.0 (SDL 2.24.2, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


В такой архитектуре, где выход первой нейронной сети бинаризуется и подается на вход второй нейронной сети, обратное распространение ошибки становится нетривиальной задачей из-за непрерывно-дискретного перехода. В стандартной ситуации обратное распространение ошибки основывается на непрерывности функций, что позволяет вычислять производные и обновлять веса через градиенты. Бинаризация, являясь дискретной операцией, нарушает этот процесс.

Для решения этой проблемы обычно применяют метод, называемый **Straight-Through Estimator (STE)**. Этот метод позволяет приблизить градиенты через бинаризацию, чтобы всё равно можно было провести обратное распространение ошибки.

Вот как это работает:

1. **Прямой проход (forward pass)**:
    - Первая нейронная сеть принимает входные данные и вычисляет свои выходные значения с сигмоидальной активацией.
    - Эти значения бинаризуются (например, с помощью порога 0.5).
    - Бинаризованные значения подаются на вход второй нейронной сети, которая решает задачу регрессии.

2. **Обратный проход (backward pass)**:
    - Вторая нейронная сеть вычисляет градиенты своей ошибки и обновляет свои веса стандартным способом.
    - Градиенты ошибки передаются обратно к выходу первой сети.
    - При обратном проходе через бинаризацию используется метод STE: мы принимаем производную бинаризации как производную сигмоидальной функции активации, что является приближением. То есть, производная сигмоида используется вместо нулевой производной бинаризации.

В коде это можно представить следующим образом:

```python
import torch
import torch.nn as nn
import torch.optim as optim

class FirstNetwork(nn.Module):
    def __init__(self):
        super(FirstNetwork, self).__init__()
        self.fc = nn.Linear(input_size, 8)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

class SecondNetwork(nn.Module):
    def __init__(self):
        super(SecondNetwork, self).__init__()
        self.fc = nn.Linear(8, 1)
    
    def forward(self, x):
        x = self.fc(x)
        return x

class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.first_network = FirstNetwork()
        self.second_network = SecondNetwork()
    
    def forward(self, x):
        x = self.first_network(x)
        x = (x >= 0.5).float()  # бинаризация с порогом 0.5
        x = self.second_network(x)
        return x

# Создание модели и определение функции потерь и оптимизатора
model = CombinedModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучающий цикл
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Обратный проход и обновление весов
        loss.backward()
        optimizer.step()
```

При таком подходе обратное распространение ошибки будет работать корректно, несмотря на присутствие бинаризации, благодаря использованию метода STE, который позволяет приближенно вычислять градиенты через дискретные шаги.

# Пример на TF

Для реализации описанного подхода с использованием TensorFlow, мы создадим две нейронные сети: первая сеть будет иметь 8 выходных нейронов с сигмоидальной функцией активации, а вторая сеть будет иметь один выходной нейрон для задачи регрессии. Мы будем использовать метод Straight-Through Estimator (STE) для выполнения обратного распространения через бинаризацию.

Вот пример кода на TensorFlow:

```python
import tensorflow as tf

class FirstNetwork(tf.keras.Model):
    def __init__(self):
        super(FirstNetwork, self).__init__()
        self.dense = tf.keras.layers.Dense(8, activation='sigmoid')
    
    def call(self, inputs):
        return self.dense(inputs)

class SecondNetwork(tf.keras.Model):
    def __init__(self):
        super(SecondNetwork, self).__init__()
        self.dense = tf.keras.layers.Dense(1)
    
    def call(self, inputs):
        return self.dense(inputs)

class CombinedModel(tf.keras.Model):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.first_network = FirstNetwork()
        self.second_network = SecondNetwork()
    
    def call(self, inputs):
        x = self.first_network(inputs)
        # Бинаризация с порогом 0.5
        x = tf.where(x >= 0.5, 1.0, 0.0)
        return self.second_network(x)

# Функция потерь и оптимизатор
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

# Создание модели
model = CombinedModel()

# Обучающий цикл
@tf.function
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        outputs = model(inputs)
        loss = loss_fn(targets, outputs)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Пример данных
import numpy as np
input_size = 10
num_samples = 1000

x_train = np.random.random((num_samples, input_size)).astype(np.float32)
y_train = np.random.random((num_samples, 1)).astype(np.float32)

# Обучение модели
num_epochs = 100
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

for epoch in range(num_epochs):
    epoch_loss = 0
    for inputs, targets in dataset:
        loss = train_step(inputs, targets)
        epoch_loss += loss.numpy()
    
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataset)}")
```

В этом коде:

1. Мы создали две отдельные сети, `FirstNetwork` и `SecondNetwork`.
2. Мы объединили эти сети в модель `CombinedModel`, где выходы первой сети бинаризуются перед подачей на вход второй сети.
3. Использовали функцию `tf.where` для бинаризации выходов первой сети.
4. Реализовали обучающий цикл с помощью `tf.GradientTape` для вычисления и применения градиентов, что позволяет обратное распространение ошибки через бинаризацию, используя STE.

Этот подход обеспечивает корректное обучение нейронной сети с бинаризацией промежуточных выходов.

In [2]:
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'